In [ ]:
import math
import csv
import numpy as np
import cv2
from scipy.io import loadmat

class DescritoresHaralick:
	def __init__(self, matrix: list) -> None:
		self.matrix = matrix
		self.rows = len(matrix)
		self.columns = len(matrix[0])

	def segundo_momento_angular(self) -> float:
		result = 0

		for i in range(0, self.rows):
			for j in range(0, self.columns):
				result += math.pow(self.matrix[i][j], 2)

		return result

	def entropia(self) -> float:
		result = 0

		for i in range(0, self.rows):
			for j in range(0, self.columns):
				if(self.matrix[i][j] == 0):
					continue
				result += (self.matrix[i][j] * math.log2(self.matrix[i][j]))

		return -result

	def contraste(self) -> float:
		result = 0

		for i in range(0, self.rows):
			for j in range(0, self.columns):
				if(self.matrix[i][j] == 0):
					continue
				result += (math.pow(i-j, 2)*self.matrix[i][j])

		return result

	def homogeneidade(self) -> float:
		result = 0

		for i in range(0, self.rows):
			for j in range(0, self.columns):
				if(self.matrix[i][j] == 0):
					continue
				result += ((1/(1 + math.pow(i-j,2))) * self.matrix[i][j])
				
		return result

	def correlacao(self):
		altura, largura = self.matrix.shape
		I, J = np.ogrid[0:altura, 0:largura]

		mi_i = np.sum(I * self.matrix)
		mi_j = np.sum(J * self.matrix)

		sigma_i = np.sqrt(np.sum((I - mi_j) ** 2 * self.matrix))
		sigma_j = np.sqrt(np.sum((J - mi_j) ** 2 * self.matrix))

		return np.sum(((I - mi_i) * (J - mi_j) * self.matrix) / (sigma_i * sigma_j))
	
def matriz_concorrencia(image, distance, angle):
    image_array = np.asarray(image)
    height, width = image_array.shape[:2]
    biggest = np.amax(image)
    matrix = np.zeros((biggest+1, biggest+1), dtype=np.float64)
    total = 0	
	
    if angle == 0:
        total = height * (width-distance)
        for i in range(0, height):
            for j in range(0, width-distance):
                matrix[image[i][j]][image[i][j+distance]] += 1
    elif angle == 90:
        total = (height-distance) * width
        for i in range(distance, height):
            for j in range(0, width):
                matrix[image[i][j]][image[i-distance][j]] += 1
    elif angle == 180:
       total = height * (width-distance)
       for i in range(0, height):
            for j in range(distance, width):
                matrix[image[i][j]][image[i][j-distance]] += 1
    elif angle == 270:
       total = (height-distance) * width
       for i in range(0, height-distance):
            for j in range(0, width):
                matrix[image[i][j]][image[i+distance][j]] += 1
                
    for i in range(0, biggest+1):
        for j in range(0, biggest+1):
            matrix[i][j] = matrix[i][j]/total
            pass

    return matrix

images = ['00', '23', '48', '68', '75', '144']
type = 'black'

with open(f'src/data/{type}/descritores.csv', mode='w', newline='') as f:
	writer = csv.writer(f)

	for imageName in images:
		distances = [1, 2]
		angles = [0, 90, 180, 270]
		matrix = []
		image = cv2.cvtColor(cv2.imread(f'src/{type}/{imageName}h.jpg'), cv2.COLOR_RGB2GRAY)

		for distance in distances:
			for angle in angles:
				
				matrix.append(matriz_concorrencia(image, distance, angle))

		matrix = np.sum(matrix, axis = 0) / len(matrix)
		matrix = matrix[1:-1, 1:-1]

		descritores_de_haralick = DescritoresHaralick(matrix)
		
		writer.writerow([
			imageName, descritores_de_haralick.segundo_momento_angular(), descritores_de_haralick.entropia(),
			descritores_de_haralick.correlacao(), descritores_de_haralick.contraste(), descritores_de_haralick.homogeneidade()
		])